# Deploy e Serving de Modelos


## Objetivo
1. Realizar deploy local do modelo para servir predições.
2. Executar inferências online e em batch.
3. Comparar o tempo de resposta e o uso de memória para as inferências.

⚠️ **ATENÇÃO: ME EXECUTE NO COLAB COM GPU OU LOCALMENTE COM GPU**

In [ ]:
# Esse comando pode demorar um pouco para rodar, mas é só na primeira vez ;)
!pip install -q transformers==4.50.3 pandas==2.2.2 torch==2.6.0\
 datasets==3.5.0 pillow==11.1.0 --progress-bar off

In [ ]:
# Verificando as configs da GPU
!nvidia-smi

In [ ]:
# Manipulação e visualização de dados
import pandas as pd
import numpy as np
import time
import psutil

# Importando modelo de classificação para imagens
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from datasets import load_dataset

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# prompt: Get GPU using cuda for pytorch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Baixando a modelo e dados para o nosso experimento

In [ ]:
dataset = load_dataset("huggingface/cats-image")
amostra = dataset["test"]["image"][-1]

In [ ]:
amostra

In [ ]:
# Paper: https://paperswithcode.com/method/resnet
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

inputs = image_processor(amostra, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

### Baixando um dataset um pouco maior e testando mais uma predição

In [ ]:
dataset = load_dataset("microsoft/cats_vs_dogs")

In [ ]:
amostra = dataset["train"][-1]["image"]
amostra

In [ ]:
inputs = image_processor(amostra, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

In [ ]:
# Movendo o modelo para a GPU
_ = model.to(device)

## Realizando o Deploy Local do Modelo

In [ ]:
N_SAMPLES = 1_000

# Selecionando algumas amostras
dados_teste = dataset["train"][:N_SAMPLES]["image"]

### Inferência em Batch

In [ ]:
# Medir o tempo de inferência em batch
inputs = image_processor(dados_teste, return_tensors="pt")
print(f'Shape inputs: {inputs["pixel_values"].shape}')
inicio_batch = time.time()
with torch.no_grad():
    predictions = model(**inputs.to(device))
predicted_labels = predictions.logits.cpu().argmax(-1).numpy()
fim_batch = time.time()

tempo_batch = fim_batch - inicio_batch
print(f"Resultado predição: {predicted_labels.shape}")
print(f"Tempo de inferência em batch: {tempo_batch:.4f} segundos")

### Inferência Online

In [ ]:
# Medir o tempo de inferência online
tempos_online = []

sample = image_processor(dados_teste[0:1], return_tensors="pt")
print(f'Shape inputs: {sample["pixel_values"].shape}')

for i in range(N_SAMPLES):  # Inferência para N amostras
    inicio_online = time.time()
    inputs = image_processor(dados_teste[i:i+1], return_tensors="pt").to(device)
    with torch.no_grad():
        predictions = model(**inputs)
    predicted_labels = predictions.logits.cpu().argmax(-1).numpy()
    fim_online = time.time()
    tempos_online.append(fim_online - inicio_online)

In [ ]:
series = pd.Series(tempos_online)
series.describe()

In [ ]:
series.sum()

In [ ]:
series.hist(bins=50)